In [ ]:
!conda info

In [ ]:
import GPUtil
GPUtil.getAvailable()

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import logging
import warnings
from typing import Callable, Optional, Union
import os, argparse

import numpy as np
import torch
import torch_dct as dct
import torch.fft as fft

from scipy import integrate
import time
import pickle
from torch.utils.data import Dataset

logger = logging.getLogger("iesolver")
logger.setLevel(logging.WARNING)#(logging.DEBUG)

import matplotlib.pyplot as plt


import torchcubicspline
from torchcubicspline import(natural_cubic_spline_coeffs, 
                             NaturalCubicSpline)
                             
from torchdiffeq import odeint


from spectral_source.integrators import MonteCarlo 
mc = MonteCarlo()

from spectral_source.spectral_integrator import Chebyshev_nchannel, spectral_integrator_nchannels, spectral_integrator, Chebyshev
from spectral_source.spectral_ie_solver import Spectral_IE_solver
from spectral_source.spectral_experiments import spectral_experiment 

from spectral_source.utils import fix_random_seeds,to_np

if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"
   
    
parser = argparse.ArgumentParser(description='Neural IE')
parser.add_argument('-root_path', metavar='DIR', default='',
                    help='path to dataset')
parser.add_argument('-dataset-name', default='stl10',
                    help='dataset name', choices=['acrobot_dataset'])

parser.add_argument('-j', '--workers', default=12, type=int, metavar='N',
                    help='number of data loading workers (default: 32)')
parser.add_argument('--epochs', default=3000, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('-b', '--batch_size', default=20, type=int,
                    metavar='N',
                    help='mini-batch size (default: 256), this is the total '
                         'batch size of all GPUs on the current node when '
                         'using Data Parallel or Distributed Data Parallel')
parser.add_argument('--lr', '--learning-rate', default=1e-4, type=float,
                    metavar='LR', help='initial learning rate', dest='lr')
parser.add_argument('--wd', '--weight-decay', default=1e-4, type=float,
                    metavar='W', help='weight decay (default: 1e-4)',
                    dest='weight_decay')
parser.add_argument('--seed', default=None, type=int,
                    help='seed for initializing training. ')
parser.add_argument('--disable-cuda', action='store_true',
                    help='Disable CUDA')
parser.add_argument('--fp16-precision', action='store_true',
                    help='Whether or not to use 16-bit precision GPU training.')

parser.add_argument('--out_dim', default=128, type=int,
                    help='feature dimension (default: 128)')
parser.add_argument('--log-every-n-steps', default=100, type=int,
                    help='Log every n steps')
parser.add_argument('--temperature', default=0.07, type=float,
                    help='softmax temperature (default: 0.07)')
parser.add_argument('--n-views', default=2, type=int, metavar='N',
                    help='Number of views for contrastive learning training.')
parser.add_argument('--gpu-index', default=0, type=int, help='Gpu index.')
parser.add_argument('--model', default='simclr', choices=['simclr','lipschitz_simclr','vae','gan'], 
                    help='Models to be used')
parser.add_argument('--mode', default='train', choices=['train','evaluate'], 
                    help='Set to ''evaluate'' if inference is desired')
parser.add_argument('--training_split', default=0.25,type=float, 
                    help='Fraction of the samples that will be used for validation')
parser.add_argument('--resume_from_checkpoint', default=None, 
                    help='Give string to run number. Ex: "run12"')
parser.add_argument('--plot_freq', default=1, type=int,help='')
parser.add_argument('--experiment_name', default=None,help='')
    
    

In [ ]:
import math
import numpy as np
from IPython.display import clear_output
from tqdm import tqdm_notebook as tqdm

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.color_palette("bright")
import matplotlib as mpl
import matplotlib.cm as cm

import torch
from torch import Tensor
from torch import nn
from torch.nn  import functional as F
from torch.autograd import Variable

use_cuda = torch.cuda.is_available()

In [ ]:
use_cuda

In [ ]:
from functools import reduce
from spectral_source import kernels, integrators 
from spectral_source.utils import to_np
import random
import pickle
import time

In [ ]:
args = parser.parse_args("")
args.model='spectral'
args.mode='train'
#args.mode='evaluate'
args.dataset_name = 'spectral_integral_equations'
args.seed = 7
args.experiment_name = 'IE_spirals'
args.plot_freq = 100
args.device = device
args.num_dim_plot = 2
args.lr = 1e-3
args.min_lr=1e-6
args.T_max = 51
args.plat_patience = 10
args.factor = 0.1
# args.temperature=0.001
#args.lr_scheduler = 'ReduceLROnPlateau'
args.lr_scheduler = 'CosineAnnealingLR'
#args.resume_from_checkpoint = 'run19'
fix_random_seeds(args.seed)
args.perturbation_to_obs0=None
args.training_split=0.2
args.smoothing_factor= .5
args.n_batch = 16
args.max_deg = 8  #maximum degree for Chebyshev polynomials used for the expansion
args.patience = 200
args.support_tensors=False
args.support_test=False
args.dim = 2
args.time_points = 100
args.f_nn = True
args.integrator_nn = False
args.chebyshev = True
args.max_iterations = 3
args.interpolation = False
args.channels = 2
args.N_MC = 1000

In [ ]:
from spectral_source.spectral_functions import F_NN, F_func

In [ ]:
Data = pickle.load(open("spectral_source/50_IE_Spirals.pkl","rb"))
Data = Data.to(device)

In [ ]:
Data.shape

In [ ]:
if args.mode == 'train':
    Data = Data[:-96,...].to(device)
if args.mode == 'evaluate':
    Data = Data[-96:,...].to(device)

In [ ]:
if args.interpolation and args.mode=='train':
    Data = Data[:,::2,:]

In [ ]:
Data.shape

In [ ]:
args.ts_integration = None

In [ ]:
nCheb = Chebyshev_nchannel(max_deg=args.max_deg,n_batch=2,N_mc=args.N_MC,device=device)

In [ ]:
def interpolator(time,obs):
        x = time
        y = obs
        coeffs = natural_cubic_spline_coeffs(x, y)
        interpolation = NaturalCubicSpline(coeffs)

        def output(point:torch.Tensor):
            return interpolation.evaluate(point)

        return output

In [ ]:
times = torch.linspace(-1,1,100).to(device)

In [ ]:
if args.interpolation:
    times = times[::2]

In [ ]:
def func_data(x):
    
    return interpolated_func(x)

In [ ]:
times.shape

In [ ]:
if args.interpolation is False:
    init_ = 50
else:
    init_ = 25

In [ ]:
X = torch.Tensor([]).to(device)
for i in range(0,Data.shape[0],2):
    interpolated_func = interpolator(times[::init_],Data[i:i+2,::init_,...])

    X_ = nCheb.expand(lambda x: interpolated_func(x))
    X = torch.cat([X,X_],0)
    del X_
print(X.shape)

In [ ]:
interpolated_func = interpolator(times,Data[:2,...])
y_spir = interpolated_func(times)

In [ ]:
y_spir.shape

In [ ]:
plt.scatter(to_np(y_spir)[0,:,0],to_np(y_spir)[0,:,1])

In [ ]:
reconstruction_func = nCheb.inverse(X[:2,...])

In [ ]:
reconstruction = reconstruction_func(times)

In [ ]:
torch.abs(reconstruction - y_spir[...])

In [ ]:
torch.abs(reconstruction - y_spir[...]).min(), torch.abs(reconstruction - y_spir[...]).max(), torch.abs(reconstruction - y_spir[...]).mean()

In [ ]:
plt.scatter(to_np(reconstruction)[0,:,0],to_np(reconstruction)[0,:,1])

In [ ]:
spectral_coeff = X

In [ ]:
spectral_integrator = spectral_integrator_nchannels(
                      [args.n_batch]+[X.shape[1]]+[X.shape[i] for i in range(1,len(X.shape))],device=device
                      )

In [ ]:
spectral_coeff.shape

In [ ]:
spectral_coeff = spectral_coeff.detach()

In [ ]:
#model = F_NN(2*(args.max_deg+1),2*args.max_deg,2*args.max_deg,[4]).to(device)
model = F_NN(2*(args.max_deg+1),2*args.max_deg,2*args.max_deg,[100,100,100]).to(device)

In [ ]:
f_func = F_func(2,2,[100,100,100]).to(device)

In [ ]:
times = torch.linspace(-1,1,100).to(device)
if args.interpolation:
    times = times[::2]

In [ ]:
nCheb = Chebyshev_nchannel(max_deg=args.max_deg,n_batch=args.n_batch,N_mc=args.N_MC,device=device)

In [ ]:
p_shapes = []
flat_parameters = []
for p in model.parameters():
    p_shapes.append(p.size())
    flat_parameters.append(p.flatten())
all_parameters = torch.cat(flat_parameters)

In [ ]:
all_parameters.shape

In [ ]:
if args.f_nn is True:    
    p_shapes = []
    flat_parameters = []
    for p in f_func.parameters():
        p_shapes.append(p.size())
        flat_parameters.append(p.flatten())
    all_parameters = torch.cat(flat_parameters)
print(all_parameters.shape)

In [ ]:
times.shape

In [ ]:
spectral_experiment(model, Data, spectral_coeff, interpolator, times, args, chebyshev_transform=nCheb, cosine_transform=None, spectral_integrator=spectral_integrator, f_func = f_func)
